## Get all html files:

- Printing all html files from the client folder

In [ ]:
import glob,os

angular_path = "" # path to Angular "client" folder
exclude = (["node_modules"])

os.chdir(angular_path)

for root, dirs, files in os.walk(".", topdown=True):
    dirs[:] = [d for d in dirs if d not in exclude]
    for name in files:
        if name.endswith(".html"):
            print(os.angular_path.join(root, name))

## Get raw html:

- printing out the raw html from files

In [ ]:
import re
import glob,os

angular_path = "" # path to Angular project folder
exclude = (["node_modules"])

os.chdir(angular_path)

for root, dirs, files in os.walk(".", topdown=True):
    dirs[:] = [d for d in dirs if d not in exclude]
    for filename in files:
        if filename.endswith(".html"):
            with open(os.angular_path.join(root, filename)) as f:
                lines = f.readlines()
                print(lines)

## Parsing HTML and extract data in readable format:

- Getting all html elements into tuples and printing them out

In [ ]:
import re
import glob,os

from html.parser import HTMLParser
from html.entities import name2codepoint

angular_path = "" # path to Angular project folder
exclude = (["node_modules"])

os.chdir(angular_path)

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)
        for attr in attrs:
            print("Attr     :", attr)

    def handle_endtag(self, tag):
        print("End tag  :", tag)

    def handle_data(self, data):
        print("Data     :", data)

    def handle_comment(self, data):
        print("Comment  :", data)

    def handle_entityref(self, name):
        c = chr(name2codepoint[name])
        print("Named ent:", c)

    def handle_charref(self, name):
        if name.startswith('x'):
            c = chr(int(name[1:], 16))
        else:
            c = chr(int(name))
        print("Num ent  :", c)

    def handle_decl(self, data):
        print("Decl     :", data)

parser = MyHTMLParser()

for root, dirs, files in os.walk(".", topdown=True):
    dirs[:] = [d for d in dirs if d not in exclude]
    for filename in files:
        if filename.endswith(".html"):
            with open(os.angular_path.join(root, filename), "r", encoding="utf8") as file:
                for line in file:
                    if "i18n" in line:
                        feed = parser.feed(line)
                        print(feed)

### Usable GUIDs:

- Prints out GUIDs which can be inserted to create new translation keys

In [ ]:
import uuid

number = 0

for number in range(100):
    print(uuid.uuid4().hex)
    print(" ")

### Parsing JSON file and return key or value:

- Used as a checking tool to get either key or value of json file

In [ ]:
import json
import re

json_path = "" # path to json file

def get_key_by_value(dict_of_elements, value_to_find):
    #listOfKeys = list() 
    list_of_items = dict_of_elements.items()
    
    for item  in list_of_items:
        if item[1] == value_to_find:
            #listOfKeys.append(item[0]) # if multiple items are returned, implement list
            return item[0]

with open(path, encoding='utf-8') as f:
    json_dict = json.load(f)
    
    print("Value:")
    # INSERT KEY HERE:
    value = json_dict.get("")
    
    print(value)
    print(" ")
    
    # INSERT VALUE HERE:
    key = get_key_by_value(json_dict, "")
    
    print("Key:")
    print(key)

## One script to rule them all:

- This script goes through all files the ends with .html in the directory line-by-line (not multiline)
- In each line, the script compares the 'content string' enclosed in the html tag (ex. < p >content string< /p >) containing the 'i18n' attribute.
- If and only if the script can match the 'content string' with a value from the json file, the 'content string' is replaced

#### Notes:
- This solution is not ideal as the attributes, attribute names, and tags have to be statically written in the script, instead this script could be rewritten into getting the tags and attribute names with a regular expressions or something similar

In [ ]:
import re
import glob,os
import fileinput
import uuid
import re
import json
from bs4 import BeautifulSoup as bs

angular_path = ""  # path to Angular project folder
exclude = (["node_modules"])

json_path = "" # path to json file which is containing the translations

os.chdir(angular_path)

old_attrs = (' i18n-title', # list of i18n attributes
             ' i18n-aria-label',
             ' i18n-error',
             ' i18n-placeholder',
             ' i18n')

old_attr_names = ('="Account|Description"', # list of the names of the attributes
                  '="Account|Placeholder"',
                  '="Account|Header"', 
                  '="Account|Column in table"')

tags_list = ('h', # list of html tags to look for
             'p',
             'li',
             'ul',
             'a',
             'span',
             'div',
             'button',
             'mat-button',
             'mat-error',
             'mat-select',
             'mat-sort-header',
             'mat-option',
             'mat-header-cell',
             'mat-radio-button',
             'mat-checkbox',
             'mat-icon')

def get_key_by_value(dict_of_elements, value_to_find):
    list_of_items = dict_of_elements.items()
    for item  in list_of_items:
        if item[1] == value_to_find:
            return item[0]

def join_strings(string_list):
    return ''.join(string for string in string_list)

# Loads the JSON data into a Dictionary:
with open(json_path, encoding='utf-8') as f:
    json_dict = json.load(f)

# Parses directories for files:
for root, dirs, files in os.walk(".", topdown=False):
    dirs[:] = [d for d in dirs if d not in exclude]
    
    # Parses each file:
    for filename in files:
        if filename.endswith(".html"):
            with fileinput.FileInput(os.angular_path.join(root, filename), inplace=True) as file:
                for line in file:
                    soup = bs(line, "html.parser")
                    
                    if 'i18n' in line:
                        for tags in tags_list:
                            used_tags = soup.find(tags)
                            
                            if used_tags is not None:
                                old_content = used_tags.get_text()
                                
                                matched_key = get_key_by_value(json_dict, old_content)

                                if matched_key is not None:
                                    string_list = ('{{ \'', matched_key, '\' | translate }}')
                                    new_content = join_strings(string_list)
                                    
                                    for old_attr in old_attrs:
                                        line = line.replace(old_attr, "")
                        
                                    for old_attr_name in old_attr_names:
                                        line = line.replace(old_attr_name, "")
                                else:
                                    continue
                                
                                line = line.replace(old_content, new_content) # does not work if old_content is None
                                
                            else:
                                continue
                    print(line, end="")